In [1]:
import pandas as pd
from pathlib import Path

from climada.hazard import Hazard
from climada.engine import ImpactCalc
from climada.entity.exposures.litpop import LitPop

import aggr_functions as fct

DATA_PATH = Path('/cluster/work/climate/carmenst/Data/Wildfires')

dir_reg_layer = Path(dir_data+'/0_REGIONAL_LAYERS')

In [2]:
haz_file = DATA_PATH+'/Hazard/150arcsec_seasonal_global_wf_2002-2025.hdf'
haz_wf = Hazard.from_hdf5(haz_file)
years = haz_wf.event_name
haz_wf

In [2]:
from climada.util.api_client import Client
client = Client()
exp_pop = client.get_litpop(None, (0,1))
exp_pc = client.get_litpop(None)

In [3]:
exposure_dir = Path(dir_data+'/Exposure')
exp_files = list(exposure_dir.glob("*.hdf5")) 
exp_files

[PosixPath('/cluster/work/climate/carmenst/StateOfWF/data/Exposure/LitPop_150arcsec_global_remapped.hdf5'),
 PosixPath('/cluster/work/climate/carmenst/StateOfWF/data/Exposure/LitPop_pop_150arcsec_remapped.hdf5')]

In [4]:
impf_set = fct.linear_impf()

In [5]:
for exposure_file in exp_files:
    
    exposure = LitPop.from_hdf5(exposure_file)
    exp_name =  exposure.fin_mode +'-'+ str(exposure.ref_year)
    print(exp_name)
    impact = ImpactCalc(exposure, impf_set, haz_wf).impact(save_mat=True) 
    
    
    # REGIONAL AGGREGATION
    #fct.wrap_reg_aggr(dir_reg_layer, impact, years, dir_data, exp_name)
    fct.wrap_reg_aggr(dir_reg_layer, haz_wf, impact, dir_data, exp_name)
    
    # GLOBAL AGGREGATION
    region_layer = 'global'
    output_file_name = dir_data+f'/Results/{exp_name}/seasonal_impacts_{exp_name}_{region_layer}.csv'
    output_file_path = Path(output_file_name)
    # Skip if file already exists
    if output_file_path.exists():
        print(f"Skipping {region_layer} — output file already exists.")
        continue
    df_global = fct.global_df(impact, years)
    df_global.to_csv(output_file_name, index=False)

pc-2018
Continental_Biomes
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/Continental_Biomes/wwf_terr_ecos_dissolved_to_continental_biomes.shp
GFED_regions
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/GFED_regions/GFED_basis_regions.shp
Ecoregions
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/Ecoregions/wwf_terr_ecos.shp
GADM_UCDavis_L1
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/GADM_UCDavis_L1/GADML1.shp
Continents
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/Continents/World_Continents.shp
IPCC_WGI_regions
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/IPCC_WGI_regions/IPCC-WGI-reference-regions-v4_land_SOW.shp
Biomes
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/Biomes/wwf_terr_ecos_dissolved_to_biomes.shp
Countries
/cluster/work/climate/carmenst/StateOfWF/data/0_REGIONAL_LAYERS/Countries/GCP_Country_Domains_v1.6_ed.shp
RECCAP2_SHP
/cluster/work/climate/ca